<a href="https://colab.research.google.com/github/javier-jaime/Route-Recommender/blob/main/Distance_Matrix_with_OSRM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Import necessary Libraries
import pandas as pd
# import googlemaps

from google.colab import drive
from google.colab import files

In [ ]:
# Mount Google Drive

drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#Read Excel file into dataframe
cities = pd.read_csv('drive/MyDrive/Colab_Data/Western_cities.csv', index_col=0)
print(len(cities))
cities

30


,Destination Province,Destination City,Latitude,Longitude
13,BC,ABBOTSFORD,49.052116,-122.329479
0,AB,ACHESON,53.545914,-113.771576
1,AB,AIRDRIE,51.285970,-114.010620
2,AB,BALZAC,51.215611,-114.004558
14,BC,BURNABY,49.243380,-122.972545
3,AB,CALGARY,51.046095,-114.065465
15,BC,DELTA,49.084626,-123.057938
4,AB,EDMONTON,53.535411,-113.507996
16,BC,ERRINGTON,49.289463,-124.370630
5,AB,GRANDE PRAIRIE,55.171080,-118.794987


 From OSRM (Open Source Routing Machine)
 https://github.com/Project-OSRM/osrm-backend/wiki/Server-api

In [ ]:
# Test the API with the coordinates from Calgary and Edmonton
route_uri = 'http://router.project-osrm.org/route/v1/driving/'
loc_pair = '-114.065465,51.046095;-113.507996,53.535411'
result = pd.read_json( route_uri + loc_pair,lines=True)
distance = result['routes'][0][0]['distance']
duration = result['routes'][0][0]['duration']
print("The distance is",distance/1000,"Kms and the duration is",duration/60,"minutes")

The distance is 297.675 Kms and the duration is 214.165 minutes


In [ ]:
# Create a new dataframe for the Distance Matrix
matrix = pd.DataFrame()
origin = []
destination = []
durations = []
distances = []

# Loop trough cites to fill the Distance Matrix
for i in range(len(cities)):
  for j in range(i+1, len(cities)):
    origin.append(cities['Destination City'][i])
    LatO = cities['Latitude'][i]
    LonO = cities['Longitude'][i]
    destination.append(cities['Destination City'][j])
    LatD = cities['Latitude'][j]
    LonD = cities['Longitude'][j]
    
    # Location pair of coordinates
    loc_pair = str(LonO) + ',' + str(LatO) + ';' + str(LonD) + ',' + str(LatD)

    #pass the location pairs to the OSRM API, read to json and extract routs and duration
    result = pd.read_json( route_uri + loc_pair,lines=True)
    distance = result['routes'][0][0]['distance']/1000
    duration = result['routes'][0][0]['duration']/60

    #Print results as they are coming
    print('From ',cities['Destination City'][i],' to ',cities['Destination City'][j],' is ',distance, 'Kms')
    
    #append result to list
    distances.append(distance)
    durations.append(duration)

matrix['Origin'] = origin
matrix['Destination'] = destination
matrix['Distance (Kms)'] = distances
matrix['Duration (Mins)'] = durations

From  ACHESON  to  AIRDRIE  is  275.60720000000003 Kms
From  ACHESON  to  BALZAC  is  283.66970000000003 Kms
From  ACHESON  to  CALGARY  is  306.5885 Kms
From  ACHESON  to  EDMONTON  is  20.6004 Kms
From  ACHESON  to  GRANDE PRAIRIE  is  441.03490000000005 Kms
From  ACHESON  to  LETHBRIDGE  is  511.0029 Kms
From  ACHESON  to  MEDICINE HAT  is  568.0886999999999 Kms
From  ACHESON  to  OKOTOKS  is  344.3179 Kms
From  ACHESON  to  RED DEER COUNTY  is  182.4985 Kms
From  ACHESON  to  ROCKY VIEW COUNTY  is  266.5219 Kms
From  ACHESON  to  SHERWOOD PARK  is  39.895 Kms
From  ACHESON  to  ST. ALBERT  is  20.1237 Kms
From  ACHESON  to  ABBOTSFORD  is  1076.7472 Kms
From  ACHESON  to  BURNABY  is  1129.6679 Kms
From  ACHESON  to  DELTA  is  1135.427 Kms
From  ACHESON  to  ERRINGTON  is  1282.4538 Kms
From  ACHESON  to  KAMLOOPS  is  787.3331 Kms
From  ACHESON  to  KELOWNA  is  899.2105 Kms
From  ACHESON  to  LADYSMITH  is  1219.316 Kms
From  ACHESON  to  LANGLEY  is  1104.0285 Kms
From  ACHESON

In [ ]:
matrix.head()

,Origin,Destination,Distance (Kms),Duration (Mins)
0,ACHESON,AIRDRIE,275.6072,200.456667
1,ACHESON,BALZAC,283.6697,204.145000
2,ACHESON,CALGARY,306.5885,221.885000
3,ACHESON,EDMONTON,20.6004,27.141667
4,ACHESON,GRANDE PRAIRIE,441.0349,311.648333


In [ ]:
# Export Distance Matrix to CSV File
matrix.to_csv('Distance_matrix.csv')
!cp Distance_matrix.csv "/content/drive/My Drive/Colab_Data/"